# Dijkstra's algorithm

## Overview
Dijkstra's algorithm finds shortest paths from a single source node to all other nodes in a weighted graph with non‑negative edge weights. It produces:
- the minimum distance from the source to every reachable node, and
- (optionally) a predecessor/previous map that allows reconstruction of the actual shortest paths.

Key idea: repeatedly finalize the node with the smallest tentative distance (greedy choice). Once a node is finalized, its shortest distance is guaranteed.

## Preconditions / assumptions
- Graph may be directed or undirected.
- All edge weights must be non‑negative (≥ 0). If negative weights exist, Dijkstra is not correct.
- Graph representation: adjacency list (neighbors with weights) is typical and efficient.

## Data structures used
- distances: map/array dist[v] = current best known distance from source to v (initialized to ∞ except dist[source] = 0).
- previous: map prev[v] = predecessor of v on the current best path (used to reconstruct paths).
- min-priority queue (min-heap) keyed by tentative distance. Each heap entry is (distance, node). Many implementations support a decrease-key operation; a simple heap implementation pushes new entries and ignores stale entries on pop.

## Pseudocode
```text
function Dijkstra(Graph, source):
    for each vertex v in Graph:
        dist[v] := ∞
        prev[v] := null
    dist[source] := 0

    Q := min-priority-queue initialized with (0, source)

    while Q not empty:
        (d, u) := pop_min(Q)
        if d > dist[u]:
            continue          // stale entry
        // u is now finalized
        for each neighbor (v, w) of u:
            alt := dist[u] + w
            if alt < dist[v]:
                dist[v] := alt
                prev[v] := u
                push (alt, v) into Q

    return dist, prev
```

Optional early stop: if you only need shortest path to a particular target, you can stop when the target node is popped (finalized).

## Correctness sketch
- Invariant: when a node u is popped with distance d = dist[u], no shorter path to u exists. Any path to u must go through nodes whose dist is ≥ d (because heap pops smallest dist first). If a shorter path existed, it would have produced a smaller tentative value and been popped earlier. Thus finalized distances are optimal.
- The relaxation step (alt = dist[u] + w) ensures distances never overestimate the true shortest paths and propagate improvements outward.

## Complexity
Assume V = |V| nodes, E = |E| edges.
- With binary heap (or Python heapq, pushing duplicates, ignoring stale): O((V + E) log V) — typically simplified to O(E log V) for connected sparse graphs.
- With Fibonacci heap (supports decrease-key): O(E + V log V).
- Using an array or linear scan for min extraction: O(V^2) — acceptable for dense graphs or small V.
- Memory: O(V + E) for adjacency lists + O(V) for dist/prev + heap.

## Implementation notes and practical tips
- Use adjacency lists: neighbors stored as (neighbor, weight) tuples. For unweighted graphs, treat weight = 1.
- If using a heap without decrease-key, allow pushing multiple (dist, node) entries and on pop skip entries where dist > dist[node].
- Keep prev pointers to reconstruct path: follow prev from target back to source and reverse.
- For integer small bounded weights, bucket-based algorithms (Dial's algorithm) can be faster.
- Dijkstra handles zero-weight edges fine.
- For graphs with negative edges use Bellman-Ford (single-source) or Johnson's algorithm (all-pairs).

## Variants & extensions
- Early termination when target popped (useful for single-source single-target).
- Multi-source shortest paths: initialize multiple sources with dist = 0 and push them into the heap.
- All-pairs shortest paths: run Dijkstra from every vertex (or use Floyd–Warshall / Johnson depending on graph).
- Path counting / tie-breaking: if you need the number of shortest paths or lexicographically smallest path, augment updates accordingly.

## Common pitfalls
- Running Dijkstra on graphs with negative edge weights leads to incorrect results.
- Forgetting to handle stale heap entries when not using decrease-key can cause wrong behavior or inefficiency; always compare popped distance against current dist[node].
- Using adjacency matrix and linear scans is fine for small V but scales poorly for sparse graphs.

## Example (concise)
- Initialize dist[source] = 0, others ∞.
- Pop source, relax neighbors — update dist and prev.
- Continue popping smallest tentative node; once popped, its distance is final.
- Reconstruct path to target by following prev pointers back to source.

Applications: routing, map navigation, network optimization, shortest-path subroutines in larger algorithms.

In [1]:
import heapq

class Graph:
    def __init__(self, directed=False):
        self.directed = directed
        self.adjacency_list = {}

    def __repr__(self):
        graph_str = ""
        for node, neighbors in self.adjacency_list.items():
            graph_str += f"{node} -> {neighbors}\n"
        return graph_str

    def add_node(self, node):
        if node not in self.adjacency_list:
            self.adjacency_list[node] = set()
        else:
            raise ValueError("Node already exists.")

    def remove_node(self, node):
        if node not in self.adjacency_list:
            raise ValueError("Node does not exist.")
        for neighbors in self.adjacency_list.values():
            neighbors.discard(node)
        del self.adjacency_list[node]

    def add_edge(self, from_node, to_node, weight=None):
        if from_node not in self.adjacency_list:
            self.add_node(from_node)
        if to_node not in self.adjacency_list:
            self.add_node(to_node)
        if weight is None:
            self.adjacency_list[from_node].add(to_node)
            if not self.directed:
                self.adjacency_list[to_node].add(from_node)
        else:
            self.adjacency_list[from_node].add((to_node, weight))
            if not self.directed:
                self.adjacency_list[to_node].add((from_node, weight))

    def remove_edge(self, from_node, to_node):
        if from_node in self.adjacency_list and to_node in self.adjacency_list[from_node]:
            self.adjacency_list[from_node].remove(to_node)
            if not self.directed and to_node in self.adjacency_list:
                if from_node in self.adjacency_list[to_node]:
                    self.adjacency_list[to_node].remove(from_node)
        else:
            raise ValueError("Edge does not exist.")

    def get_neighbors(self, node):
        return self.adjacency_list.get(node, set())

    def has_node(self, node):
        return node in self.adjacency_list

    def has_edge(self, from_node, to_node):
        if from_node in self.adjacency_list:
            return to_node in self.adjacency_list[from_node]
        return False

    def get_nodes(self):
        return list(self.adjacency_list.keys())

    def get_edges(self):
        edges = []
        for from_node, neighbors in self.adjacency_list.items():
            for to_node in neighbors:
                edges.append((from_node, to_node))
        return edges

    def bfs(self, start):
        visited = set()
        queue = [start]
        order = []
        while queue:
            node = queue.pop(0)
            if node not in visited:
                visited.add(node)
                order.append(node)
                neighbors = self.get_neighbors(node)
                for neighbor in neighbors:
                    # If edges are weighted, neighbor might be a tuple
                    if isinstance(neighbor, tuple):
                        neighbor = neighbor[0]
                    if neighbor not in visited:
                        queue.append(neighbor)
        return order

    def dfs(self, start):
        visited = set()
        stack = [start]
        order = []
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                order.append(node)
                neighbors = self.get_neighbors(node)
                for neighbor in sorted(neighbors, reverse=True):
                    if isinstance(neighbor, tuple):
                        neighbor = neighbor[0]
                    if neighbor not in visited:
                        stack.append(neighbor)
        return order


In [2]:
def dijkstra(self, start, target=None):
    """
    Compute shortest paths from start to all nodes or to a specific target.
    Returns:
        - if target is None: (distances_dict, previous_dict)
        - if target provided: (distance_to_target, path_list) or (None, []) if unreachable
    """
    # Validate that the start node exists in the graph
    if start not in self.adjacency_list:
        raise ValueError("Start node not in graph")

    # Initialize distances to infinity and previous pointers to None
    # distances: tentative shortest known distance from start to each node
    # previous: to reconstruct the shortest path tree (previous[node] = predecessor on path)
    distances = {node: float('inf') for node in self.get_nodes()}
    previous = {node: None for node in self.get_nodes()}
    distances[start] = 0

    # Min-heap priority queue stores (distance, node). We always pop the node with smallest tentative distance.
    heap = [(0, start)]
    while heap:
        # Extract node u with smallest known distance dist_u
        dist_u, u = heapq.heappop(heap)
        # If this distance is stale (we already found a better path), skip processing
        if dist_u > distances[u]:
            continue

        # Optional early exit: if we're looking for a specific target and we popped it, its shortest distance is finalized
        if target is not None and u == target:
            break

        # Relax all edges (u -> v). For each neighbor v with edge weight w:
        #   alt = dist_u + w is the distance to v via u.
        #   If alt is better than current distances[v], update distances and previous and push the new tentative distance into the heap.
        for nbr in self.get_neighbors(u):
            if isinstance(nbr, tuple):
                v, w = nbr
            else:
                # For unweighted graphs treat each edge as weight 1
                v, w = nbr, 1  # default weight for unweighted edges
            alt = dist_u + w
            # Found a shorter path to v via u
            if alt < distances[v]:
                distances[v] = alt
                previous[v] = u
                # Push updated tentative distance for v into heap. We may push multiple entries for v; stale ones are ignored above.
                heapq.heappush(heap, (alt, v))

    # If no specific target requested, return all distances and the previous map for path reconstruction
    if target is None:
        return distances, previous

    # If target requested but unreachable, indicate it
    if distances[target] == float('inf'):
        return None, []
    # Otherwise reconstruct the shortest path from start to target using the previous pointers
    path = []
    node = target
    while node is not None:
        path.append(node)
        node = previous[node]
    path.reverse()
    return distances[target], path

Graph.dijkstra = dijkstra


#### Example graph (undirected, weighted):

```
        (4)
A ───────────── B
 \             / \
  \ (2)    (1)/   \ (5)
   \         /     \
    \       /       \
     \    /          \
       C ────(8)───── D ──(6)── Z
        \             /
         \(10)    (2)/
          \         /
            \     /
               E 

```

Edges:
- A-B: 4
- A-C: 2
- B-C: 1
- B-D: 5
- C-D: 8
- C-E: 10
- D-E: 2
- D-Z: 6

Goal: run Dijkstra from A, compute all shortest distances, and show shortest path A -> Z.

Algorithm notes (concise):
- Initialize distances: dist[A]=0, others=∞. Use a min-heap of (distance, node).
- Repeatedly pop the node u with smallest tentative distance; this distance is final for u.
- For each neighbor v of u with edge weight w, compute alt = dist[u] + w. If alt < dist[v], update dist[v] and previous[v], and push (alt, v) onto the heap.
- Heap may contain stale entries for a node; skip them when popped by comparing with current dist[node].
- Optionally stop early when target is popped (finalized).

Step-by-step trace (major events):

Initial state:
- dist: {A:0, B:∞, C:∞, D:∞, E:∞, Z:∞}
- heap: [(0, A)]
- visited: {}

1) Pop (0, A). Relax neighbors B and C:
- from A: B = 0 + 4 = 4 (prev[B]=A), C = 0 + 2 = 2 (prev[C]=A)
- dist: {A:0, B:4, C:2, D:∞, E:∞, Z:∞}
- heap: [(2, C), (4, B)]
- visited: {A}

2) Pop (2, C). Relax neighbors B, D, E:
- via C -> B: alt = 2 + 1 = 3 < 4 ⇒ update B=3 (prev[B]=C)
- via C -> D: alt = 2 + 8 = 10 ⇒ D=10 (prev[D]=C)
- via C -> E: alt = 2 + 10 = 12 ⇒ E=12 (prev[E]=C)
- dist: {A:0, B:3, C:2, D:10, E:12, Z:∞}
- heap: [(3, B), (4, B) /*stale*/, (10, D), (12, E)]
- visited: {A, C}

3) Pop (3, B). Relax neighbors D (and A/C already visited):
- via B -> D: alt = 3 + 5 = 8 < 10 ⇒ update D=8 (prev[D]=B)
- dist: {A:0, B:3, C:2, D:8, E:12, Z:∞}
- heap: [(4, B) /*stale*/, (8, D), (10, D) /*stale*/, (12, E)]
- visited: {A, C, B}

4) Pop (4, B) — stale (dist[B]=3) ⇒ skip.

5) Pop (8, D). Relax neighbors E and Z (B/C already visited):
- via D -> E: alt = 8 + 2 = 10 < 12 ⇒ update E=10 (prev[E]=D)
- via D -> Z: alt = 8 + 6 = 14 ⇒ Z=14 (prev[Z]=D)
- dist: {A:0, B:3, C:2, D:8, E:10, Z:14}
- heap: [(10, D) /*stale*/, (12, E) /*stale*/, (10, E), (14, Z)]
- visited: {A, C, B, D}

6) Pop (10, D) — stale ⇒ skip.

7) Pop (10, E). Relax neighbors (all lead to visited nodes or not improve distances): no change.
- visited: {A, C, B, D, E}

8) Pop (12, E) — stale ⇒ skip.

9) Pop (14, Z). Z finalized; if we requested target=Z we can stop. Final distances obtained.

Final distances (from A):
- A: 0
- B: 3    (A -> C -> B)
- C: 2    (A -> C)
- D: 8    (A -> C -> B -> D)
- E: 10   (A -> C -> B -> D -> E)
- Z: 14   (A -> C -> B -> D -> Z)

Shortest path A -> Z (reconstructed via previous pointers):
- prev[Z] = D, prev[D] = B, prev[B] = C, prev[C] = A ⇒ path = [A, C, B, D, Z], distance = 14

Remarks / visualization tips:
- The heap may contain multiple entries for the same node; stale entries are ignored when popped (compare with current dist[node]).
- Dijkstra finalizes a node the first time it is popped from the min-heap with its current best distance.
- For this graph, the cheapest route to Z uses the multi-hop path A -> C -> B -> D -> Z rather than any direct long edge.

In [3]:

g = Graph(directed=False)

g.add_edge('A', 'B', 4)
g.add_edge('A', 'C', 2)
g.add_edge('B', 'C', 1)
g.add_edge('B', 'D', 5)
g.add_edge('C', 'D', 8)
g.add_edge('C', 'E', 10)
g.add_edge('D', 'E', 2)
g.add_edge('D', 'Z', 6)

# shortest distances from A to all nodes
distances, prev = g.dijkstra('A')
print('Distances from A:')
for node, d in distances.items():
    print(f"  {node}: {d}")

# shortest path from A to Z
distance_to_Z, path_to_Z = g.dijkstra('A', target='Z')
print('\nShortest path A -> Z:')
print('  distance =', distance_to_Z)
print('  path =', path_to_Z)

Distances from A:
  A: 0
  B: 3
  C: 2
  D: 8
  E: 10
  Z: 14

Shortest path A -> Z:
  distance = 14
  path = ['A', 'C', 'B', 'D', 'Z']


In [4]:
# Example 1: Single node (start == target)
g1 = Graph()
g1.add_node('S')
dist, path = g1.dijkstra('S', target='S')
print('Ex1 - Single node -> distance:', dist, 'path:', path)

Ex1 - Single node -> distance: 0 path: ['S']


In [5]:
# Example 2: Unweighted undirected graph (implicit weight=1)
g2 = Graph()
g2.add_edge('A', 'B')
g2.add_edge('B', 'C')
g2.add_edge('C', 'D')
dist, path = g2.dijkstra('A', target='D')
print('Ex2 - Unweighted A->D distance:', dist, 'path:', path)

Ex2 - Unweighted A->D distance: 3 path: ['A', 'B', 'C', 'D']


In [6]:
# Example 3: Weighted graph with multiple equal shortest paths
g3 = Graph()
g3.add_edge('A', 'B', 1)
g3.add_edge('A', 'C', 1)
g3.add_edge('B', 'D', 1)
g3.add_edge('C', 'D', 1)
dist, path = g3.dijkstra('A', target='D')
print('Ex3 - Multiple equal shortest paths A->D distance:', dist, 'path:', path)

Ex3 - Multiple equal shortest paths A->D distance: 2 path: ['A', 'B', 'D']


In [7]:
# Example 4: Disconnected graph where target unreachable
g4 = Graph()
g4.add_edge('A', 'B', 2)
g4.add_edge('X', 'Y', 3)
res = g4.dijkstra('A', target='Y')
print('Ex4 - Disconnected A->Y result:', res)

Ex4 - Disconnected A->Y result: (None, [])


In [8]:
# Example 5: Directed graph asymmetry
g5 = Graph(directed=True)
g5.add_edge('A', 'B', 1)
g5.add_edge('B', 'C', 2)
# Note: no edge C->A, so paths differ
dist_ac, path_ac = g5.dijkstra('A', target='C')
print('Ex5 - Directed A->C distance:', dist_ac, 'path:', path_ac)
try:
    # from C to A should be unreachable
    print('Ex5 - Directed C->A:', g5.dijkstra('C', target='A'))
except Exception as e:
    print('Ex5 - C->A raised:', e)

Ex5 - Directed A->C distance: 3 path: ['A', 'B', 'C']
Ex5 - Directed C->A: (None, [])


In [9]:
# Example 6: Zero-weight edges
g6 = Graph()
g6.add_edge('A', 'B', 0)
g6.add_edge('B', 'C', 2)
g6.add_edge('A', 'C', 5)
dist_ac, path_ac = g6.dijkstra('A', target='C')
print('Ex6 - Zero-weight path A->C distance:', dist_ac, 'path:', path_ac)

Ex6 - Zero-weight path A->C distance: 2 path: ['A', 'B', 'C']


In [10]:
# Example 7: Graph with cycles
g7 = Graph()
g7.add_edge('A', 'B', 1)
g7.add_edge('B', 'C', 1)
g7.add_edge('C', 'A', 1)
g7.add_edge('B', 'D', 2)
dist_ad, path_ad = g7.dijkstra('A', target='D')
print('Ex7 - Cyclic graph A->D distance:', dist_ad, 'path:', path_ad)

Ex7 - Cyclic graph A->D distance: 3 path: ['A', 'B', 'D']


In [11]:
# Example 8: Multiple components with an isolated node
g8 = Graph()
g8.add_edge(1, 2, 1)
g8.add_edge(2, 3, 1)
g8.add_node(4)  # isolated
distances, _ = g8.dijkstra(1)
print('Ex8 - distances from 1:', distances)
print('Ex8 - 1->4 should be unreachable:', g8.dijkstra(1, target=4))

Ex8 - distances from 1: {1: 0, 2: 1, 3: 2, 4: inf}
Ex8 - 1->4 should be unreachable: (None, [])


In [12]:
# Example 9: Larger graph typical scenario
g9 = Graph()
g9.add_edge('S', 'A', 7)
g9.add_edge('S', 'B', 2)
g9.add_edge('B', 'A', 3)
g9.add_edge('A', 'C', 4)
g9.add_edge('B', 'C', 6)
g9.add_edge('C', 'T', 1)
g9.add_edge('A', 'T', 5)
dist_st, path_st = g9.dijkstra('S', target='T')
print('Ex9 - S->T distance:', dist_st, 'path:', path_st)

Ex9 - S->T distance: 9 path: ['S', 'B', 'C', 'T']


In [13]:
# Example 10: Alternative weighted edges selecting cheaper multi-hop
g10 = Graph()
g10.add_edge('S', 'X', 10)
g10.add_edge('S', 'Y', 5)
g10.add_edge('Y', 'X', 3)
dist_sx, path_sx = g10.dijkstra('S', target='X')
print('Ex10 - S->X distance (via Y):', dist_sx, 'path:', path_sx)

Ex10 - S->X distance (via Y): 8 path: ['S', 'Y', 'X']


In [14]:
# Example 11: Error handling when start not in graph
g11 = Graph()
g11.add_edge('A', 'B', 1)
try:
    print('Ex11 - query from missing node:')
    g11.dijkstra('Missing')
except Exception as e:
    print('Ex11 - caught exception as expected:', type(e).__name__, '-', e)

Ex11 - query from missing node:
Ex11 - caught exception as expected: ValueError - Start node not in graph


In [15]:
# Example 12: Immediate neighbor target
g12 = Graph()
g12.add_edge('A', 'B', 2)
dist_ab, path_ab = g12.dijkstra('A', target='B')
print('Ex12 - A->B distance:', dist_ab, 'path:', path_ab)

Ex12 - A->B distance: 2 path: ['A', 'B']
